In [27]:
import math
from decimal import *
import numpy as np
import pygraphviz as pgv

getcontext().prec = 3

trainingData = np.genfromtxt('hw3train.txt', dtype="f8", delimiter=' ')
testData = np.genfromtxt('hw3test.txt', dtype="f8", delimiter=' ')

features = testData[0].size-1
labels = np.unique(testData[:,features])

class ID3DecisionTree:
    
    leafCounter = 0
    
    def __init__ (self, data):
        self.data = data
        self.leaf = False
        self.parent = None
        self.feature = None
        self.threshold = None
        self.pure = False
        self.number = -1
        self.left = None
        self.right = None
        self.graph = pgv.AGraph(directed=True, strict=True)
        self.graph.layout('dot')
        
        
    def __str__ (self):
        if (self.leaf == False):
            return "f"+`self.feature` + "<=" + `self.threshold`
        else:
            if self.number == -1:
                self.number = ID3DecisionTree.leafCounter
                ID3DecisionTree.leafCounter += 1
            return "leaf " + `self.number` + ": " + `self.data.shape[0]` + " label-" + `int(self.data[0][features])` +" data points."
        
    def generate(self, graph):
        
        #if the tree doesn't have a feature and threshold, create them
        if (self.feature == None) & (self.threshold == None):
            featureAndThreshold = self.findFeatureAndThreshold(self.data)
            self.feature = featureAndThreshold[0]
            self.threshold = featureAndThreshold[1]
            
        #find what data go to the left and right of the condition feature <= threshold
        (leftData, rightData) = self.splitDataByFeatureAlongThreshold(self.data, self.feature, self.threshold)
        
        
        #if either dataset is pure, make into a leaf. Else, make into a tree
        if self.getIsPure(rightData):
            self.right = ID3DecisionTree(rightData)
            self.right.pure = True
            self.right.leaf = True
#             graph.add_edge(str(self), str(self) + "->" + str(self.right))
            print("Right pure: " + `(str(self), str(self) + "->" + str(self.right))`)
        else:
            print("Right impure...")
            self.right = ID3DecisionTree(rightData)
            self.right.parent = self
            self.right.generate(graph)
            print("Right impure (finally): " + `(str(self), str(self.right))`)
            
        if self.getIsPure(leftData):
            self.left = ID3DecisionTree(leftData)
            self.left.pure = True
            self.left.leaf = True
#             graph.add_edge(str(self), str(self) + "->" + str(self.left))
            print("Left pure: " + `(str(self), str(self) + "->" + str(self.left))`)
        else:
            print("Left impure...")
            self.left = ID3DecisionTree(leftData)
            self.left.parent = self
            self.left.generate(graph)
#             graph.add_edge(str(self), str(self.right))
            print("Left impure (finally): " + `(str(self), str(self.left))`)
            
        
        graph.add_edge(str(self), str(self.right), label='No')
        graph.add_edge(str(self), str(self.left), label = 'Yes')
            
        
            
        if (self.left.pure) & (self.right.pure):
            self.pure = True
            
    def draw(self):
        self.graph.write('graph.dot')
        self.graph = pgv.AGraph('graph.dot')
        self.graph.layout('dot')
        self.graph.draw('graph.png')
        
    def traverse(self, vector):
        node = self
#         print("Given " + `vector`)
        while node.leaf == False:
#             print("Comparing node's feature " + `node.feature` + " to vector's feature " + `node.feature`)
#             print("Is vector's feature " + `node.feature` + " (" + `vector[node.feature]` + ") less than or equal to " + `node.threshold` + "?")
            
            if vector[node.feature] <= node.threshold:
#                 print("Yes. Traversing left.")
                node = node.left
            else:
#                 print("No. Traversing right.")
                node = node.right
        
#         print("Reached a leaf node. Guessing that vector has label " + `node.data[0][features]`)
        return node.data[0][features]
            
            
    def computeError(self, dataMatrix):
    
        predictedValues = []
        expectedValues = dataMatrix[:,features]
        
        for vector in dataMatrix:
            predictedValue = self.traverse(vector)
            predictedValues.append(predictedValue)
            
        errorArray = np.equal(predictedValues, expectedValues)
        errorArray = np.invert(errorArray)
        
        return np.sum(errorArray.astype(int))/expectedValues.shape[0]
        
    
    def computeEntropy(self, data):
        occurences = np.zeros(labels.size)
        
        for vector in data:
            label = vector[features]
            occurences[int(label-1)] = occurences[int(label-1)] + 1
            
        occurences = occurences/np.sum(occurences)
        occurences = np.log(occurences)
                
        entropy = -np.sum(occurences)
    
    def computeConditionalEntropy(self, data, feature, threshold):
        
        ProbabilityFeatureIsLess = float((data[:, feature] <= threshold).sum())/float(data.shape[0])
        ProbabilityFeatureIsGreater = float((data[:, feature] > threshold).sum())/float(data.shape[0])
        
        FLsThVData = data[data[:,0] <= threshold]
        FLsThVLabels = FLsThVData[:, features]
        FGrThVData = data[data[:,0] > threshold]
        FGrThVLabels = FGrThVData[:, features]
        
        PrOfEachFLsThVLabel = (np.unique(FLsThVLabels, return_counts=1)[1])/float(FLsThVData.shape[0])
        PrOfEachFGrThVLabel = (np.unique(FGrThVLabels, return_counts=1)[1])/float(FGrThVData.shape[0])
        
        EntropyGivenFLsThV = -np.sum(PrOfEachFLsThVLabel * np.log(PrOfEachFLsThVLabel))
        EntropyGivenFGrThV = -np.sum(PrOfEachFGrThVLabel * np.log(PrOfEachFGrThVLabel))
        
        conditionalEntropy = ProbabilityFeatureIsGreater*EntropyGivenFGrThV + ProbabilityFeatureIsLess*EntropyGivenFLsThV
       
        return conditionalEntropy
        
    def findFeatureAndThreshold(self, data):
        #create an array of infinitely high values for the initial entropies of the features
        conditionalEntropiesOfFeatures = np.full((features, 2), np.inf)
        
        #for each feature in the dataset
        for feature in range(features):
            
            #create potential-threshold values in between the values of that feature
            valueSet = np.unique(data[:, feature])
            sizeOfValueSet = valueSet.shape[0]-1
            valueSet = valueSet[:sizeOfValueSet] + (np.diff(valueSet)/2.0)
            
            #for each of those inbetween potential-threshold values
            for value in valueSet:
                #compute the conditional entropy
                conditionalEntropy = self.computeConditionalEntropy(data, feature, value)
#                 print("Conditional Entropy on feature " + `feaure` + " with value " + `value` + ": " + `conditionalEntropy`)
                #store lower and lower conditional entropies
                if conditionalEntropy < conditionalEntropiesOfFeatures[feature][0]:
                    conditionalEntropiesOfFeatures[feature] = (conditionalEntropy, value)
                    
        featureWithLeastConditionalEntropy = conditionalEntropiesOfFeatures[:,0].argmin()
        threshold = conditionalEntropiesOfFeatures[featureWithLeastConditionalEntropy][1]
        print(conditionalEntropiesOfFeatures[featureWithLeastConditionalEntropy][1])
        return (featureWithLeastConditionalEntropy, Decimal(threshold)/Decimal(1.00))
        
        
    def splitDataByFeatureAlongThreshold(self, data, feature, threshold):
        leftData = data[data[:,feature] <= threshold]
        rightData = data[data[:,feature] > threshold]
        return (leftData , rightData)
    
    def getIsPure(self, data):
        return  np.unique(data[:,features]).shape[0] == 1

In [28]:
decisionTree = ID3DecisionTree(trainingData)

decisionTree.computeEntropy(trainingData)

featureThreshPair = decisionTree.findFeatureAndThreshold(trainingData)
decisionTree.splitDataByFeatureAlongThreshold(trainingData, featureThreshPair[0], featureThreshPair[1])
decisionTree.getIsPure(trainingData)

decisionTree.generate(decisionTree.graph)
decisionTree.draw()

4.3
4.3
Right pure: ("f1<=Decimal('4.30')", "f1<=Decimal('4.30')->leaf 0: 1 label-1 data points.")
Left impure...
4.1
Right pure: ("f1<=Decimal('4.10')", "f1<=Decimal('4.10')->leaf 1: 1 label-1 data points.")
Left impure...
3.95
Right pure: ("f1<=Decimal('3.95')", "f1<=Decimal('3.95')->leaf 2: 1 label-1 data points.")
Left impure...
3.85
Right pure: ("f1<=Decimal('3.85')", "f1<=Decimal('3.85')->leaf 3: 1 label-1 data points.")
Left impure...
2.4
Right pure: ("f3<=Decimal('2.40')", "f3<=Decimal('2.40')->leaf 4: 2 label-3 data points.")
Left impure...
3.75
Right impure...
6.7
Right pure: ("f0<=Decimal('6.70')", "f0<=Decimal('6.70')->leaf 5: 2 label-3 data points.")
Left pure: ("f0<=Decimal('6.70')", "f0<=Decimal('6.70')->leaf 6: 4 label-1 data points.")
Right impure (finally): ("f1<=Decimal('3.75')", "f0<=Decimal('6.70')")
Left impure...
3.65
Right pure: ("f1<=Decimal('3.65')", "f1<=Decimal('3.65')->leaf 7: 3 label-1 data points.")
Left impure...
3.55
Right pure: ("f1<=Decimal('3.55')", 

In [458]:
decisionTree.computeError(testData)

0.10000000000000001

In [6]:
decisionTree.probabilityOfDataClass(testData, 1)

/Users/maxhoward/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:142: VisibleDeprecationWarning: boolean index did not match indexed array along dimension 0; dimension is 50 but corresponding boolean dimension is 5


0

In [9]:
testData.shape[1]

5

0.4